<a href="https://colab.research.google.com/github/cernavcii/MyFirstRepo/blob/main/Feature_Engineering_%26_Data_Preprocessing_Uygulama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [ ]:
df = pd.read_csv("titanic.csv")

In [ ]:
df.shape
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.columns=[col.upper() for col in df.columns]
df

,PASSENGERID,SURVIVED,PCLASS,NAME,SEX,AGE,SIBSP,PARCH,TICKET,FARE,CABIN,EMBARKED
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):

    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]

    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]

    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    # cat_cols + num_cols + cat_but_car = number of variables.
    # num_but_cat is already in cat_cols.
    # dolayısıyla *all variables will be selected with all 3 lists: cat_cols + num_cols + cat_but_car
    # num_but_cat *indicated for reporting only

    return cat_cols, cat_but_car, num_cols

**1. Feature Enginnering**

In [ ]:
# Cabin bool
df["NEW_CABIN_BOOL"] = df["CABIN"].notnull().astype("int")

In [ ]:
# Name Count
df["NEW_NAME_COUNT"] = df["NAME"].str.len()

In [ ]:
# name word count
df["NEW_NAME_WORD_COUNT"] = df["NAME"].apply(lambda x: len(str(x).split(" ")))

In [ ]:
# name dr
df["NEW_NAME_DR"] = df["NAME"].apply(lambda x: len([x for x in x.split() if x.startswith("Dr")]))


In [ ]:
# NAME TITLE
df["NEW_TITLE"] = df.NAME.str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
# Family Size
df["NEW_FAMILY_SIZE"] = df["SIBSP"] + df["PARCH"] + 1

In [ ]:
# Age_pclass
df["NEW_AGE_PCLASS"] = df["AGE"] * df["PCLASS"]

In [ ]:
# is alone
df.loc[((df['SIBSP']+ df['PARCH'])>0), 'NEW_IS_ALONE'] = 'NO'
df.loc[((df['SIBSP']+ df['PARCH'])==0), 'NEW_IS_ALONE'] = 'YES'

In [ ]:
# age level
df.loc[(df['AGE']<18), 'NEW_AGE_CAT'] = 'young'
df.loc[(df['AGE']>=18) & (df['AGE']<56), 'NEW_AGE_CAT'] = 'mature'
df.loc[(df['AGE']>=56), 'NEW_AGE_CAT'] = 'senior'

In [ ]:
# sex x age
df.loc[(df['SEX']=='male') & (df['AGE']<=21), 'NEW_SEX_CAT'] = 'youngmale'
df.loc[(df['SEX']=='male') & ((df['AGE']>21) & (df['AGE']<=50)), 'NEW_SEX_CAT'] = 'maturemale'
df.loc[(df['SEX']=='male') & (df['AGE']>50), 'NEW_SEX_CAT'] = 'seniormale'
df.loc[(df['SEX']=='female') & (df['AGE']<=21), 'NEW_SEX_CAT'] = 'youngfemale'
df.loc[(df['SEX']=='female') & ((df['AGE']>21) & (df['AGE']<=50)), 'NEW_SEX_CAT'] = 'maturefemale'
df.loc[(df['SEX']=='female') & (df['AGE']>50), 'NEW_SEX_CAT'] = 'seniorfemale'

In [ ]:
cat_cols , num_cols,cat_but_car = grab_col_names(df)

Observations: 891
Variables: 22
cat_cols: 14
num_cols: 5
cat_but_car: 3
num_but_cat: 8


In [ ]:
num_cols = [col for col in num_cols if "PASSENGERID" not in col]

**2.Outliers**

In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.10, q3=0.90):
    if dataframe[col_name].dtype not in [int, float]: # Check if the column is numeric
        return None, None # Return None if the column is not numeric
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

for col in num_cols:
    result = outlier_thresholds(df, col)
    if result != (None, None): # Check if the function returned valid limits
        print(col, result)


def replace_with_thresholds(dataframe, col_name, q1=0.10, q3=0.90):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name, q1, q3)
    if low_limit is not None and up_limit is not None:
        dataframe[col_name] = dataframe[col_name].clip(low_limit, up_limit) # Replace values outside the thresholds with the thresholds
    return dataframe

In [ ]:
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False
for col in num_cols:
    print(col, check_outlier(df, col))

NAME False
TICKET False
CABIN False


In [ ]:
for col in num_cols:
    print(col, check_outlier(df, col))

NAME False
TICKET False
CABIN False


In [ ]:
for col in num_cols:
    print(col, replace_with_thresholds(df, col))

NAME      PASSENGERID  SURVIVED  PCLASS  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  NAME     SEX   AGE  SIBSP  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...  

3.Missing Values

In [ ]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns
missing_values_table(df)

                n_miss  ratio
CABIN              687  77.10
AGE                177  19.87
NEW_AGE_PCLASS     177  19.87
EMBARKED             2   0.22


In [ ]:
df.drop("CABIN", axis=1, inplace=True)

In [ ]:
remove_cols = ["TICKET", "NAME"]
df.drop(remove_cols, axis=1, inplace=True)

In [ ]:
df["AGE"] = df["AGE"].fillna(df.groupby("NEW_TITLE")["AGE"].transform("median"))

In [ ]:
# Age_pclass
df["NEW_AGE_PCLASS"] = df["AGE"] * df["PCLASS"]
# age level
df.loc[(df['AGE']<18), 'NEW_AGE_CAT'] = 'young'
df.loc[(df['AGE']>=18) & (df['AGE']<56), 'NEW_AGE_CAT'] = 'mature'
df.loc[(df['AGE']>=56), 'NEW_AGE_CAT'] = 'senior'
# sex x age
df.loc[(df['SEX']=='male') & (df['AGE']<=21), 'NEW_SEX_CAT'] = 'youngmale'
df.loc[(df['SEX']=='male') & ((df['AGE']>21) & (df['AGE']<=50)), 'NEW_SEX_CAT'] = 'maturemale'
df.loc[(df['SEX']=='male') & (df['AGE']>50), 'NEW_SEX_CAT'] = 'seniormale'
df.loc[(df['SEX']=='female') & (df['AGE']<=21), 'NEW_SEX_CAT'] = 'youngfemale'
df.loc[(df['SEX']=='female') & ((df['AGE']>21) & (df['AGE']<=50)), 'NEW_SEX_CAT'] = 'maturefemale'
df.loc[(df['SEX']=='female') & (df['AGE']>50), 'NEW_SEX_CAT'] = 'seniorfemale'

In [ ]:
df = df.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique())<=10) else x,axis=0)

**4.Label Encodin**

In [ ]:
binary_cols = [col for col in df.columns if df[col].dtype not in [int, float] and df[col].nunique() == 2]
binary_cols

['SEX', 'NEW_IS_ALONE']

In [ ]:
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

In [ ]:
for col in binary_cols:
    df = label_encoder(df, col)

**5. Rare Encodin**

In [ ]:
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts())) # Indent this line
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                          "RATIO": dataframe[col].value_counts() / len(dataframe)}))

In [ ]:
rare_analyser(df, "SURVIVED", cat_cols)

SEX : 2
     COUNT     RATIO
SEX                 
1      577  0.647587
0      314  0.352413
EMBARKED : 3
          COUNT     RATIO
EMBARKED                 
S           646  0.725028
C           168  0.188552
Q            77  0.086420
NEW_TITLE : 17
           COUNT     RATIO
NEW_TITLE                 
Mr           517  0.580247
Miss         182  0.204265
Mrs          125  0.140292
Master        40  0.044893
Dr             7  0.007856
Rev            6  0.006734
Mlle           2  0.002245
Major          2  0.002245
Col            2  0.002245
Countess       1  0.001122
Capt           1  0.001122
Ms             1  0.001122
Sir            1  0.001122
Lady           1  0.001122
Mme            1  0.001122
Don            1  0.001122
Jonkheer       1  0.001122
NEW_IS_ALONE : 2
              COUNT     RATIO
NEW_IS_ALONE                 
1               537  0.602694
0               354  0.397306
NEW_AGE_CAT : 3
             COUNT     RATIO
NEW_AGE_CAT                 
mature         735  0.8249

In [ ]:
def rare_encoder(dataframe, rare_perc):
    # Your rare encoding logic here
    # ...
    return dataframe

df = rare_encoder(df, 0.01)

In [ ]:
df = rare_encoder(df, 0.01)

In [ ]:
df["NEW_TITLE"].value_counts()

,count
NEW_TITLE,
Mr,517
Miss,182
Mrs,125
Master,40
Dr,7
Rev,6
Mlle,2
Major,2
Col,2


**6.One-Hot Encoding**

In [ ]:
ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]

In [ ]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

In [ ]:
df = one_hot_encoder(df, ohe_cols)

In [ ]:
df.head()

,PASSENGERID,SURVIVED,SEX,AGE,FARE,NEW_CABIN_BOOL,NEW_NAME_COUNT,NEW_NAME_DR,NEW_TITLE,NEW_AGE_PCLASS,...,NEW_FAMILY_SIZE_11,NEW_AGE_CAT_mature,NEW_AGE_CAT_senior,NEW_AGE_CAT_young,NEW_SEX_CAT_maturefemale,NEW_SEX_CAT_maturemale,NEW_SEX_CAT_seniorfemale,NEW_SEX_CAT_seniormale,NEW_SEX_CAT_youngfemale,NEW_SEX_CAT_youngmale
0,1,0,1,22.0,7.2500,0,23,0,Mr,66.0,...,False,True,False,False,False,True,False,False,False,False
1,2,1,0,38.0,71.2833,1,51,0,Mrs,38.0,...,False,True,False,False,True,False,False,False,False,False
2,3,1,0,26.0,7.9250,0,22,0,Miss,78.0,...,False,True,False,False,True,False,False,False,False,False
3,4,1,0,35.0,53.1000,1,44,0,Mrs,35.0,...,False,True,False,False,True,False,False,False,False,False
4,5,0,1,35.0,8.0500,0,24,0,Mr,105.0,...,False,True,False,False,False,True,False,False,False,False


In [ ]:
df.shape

(891, 57)